In [1]:
import tvm
from tvm import te
import numpy as np

In [2]:
n = tvm.tir.const(128, "int32")
a = te.placeholder((n,), name="a")
b = te.placeholder((n,), name="b")
c = te.compute((n,), lambda i: a[i] + b[i], name="c")

sch = te.create_prim_func([a, b, c]).with_attr({"global_symbol": "add"})
ir = tvm.IRModule({"add": sch})

In [17]:
ir["add"].body

with T.block("root"):
    T.reads()
    T.writes()
    for i in range(128):
        with T.block("c"):
            v_i = T.axis.spatial(128, i)
            a = T.Buffer((128,))
            b = T.Buffer((128,))
            T.reads(a[v_i], b[v_i])
            c = T.Buffer((128,))
            T.writes(c[v_i])
            c[v_i] = a[v_i] + b[v_i]

In [7]:
ir.show()

In [11]:
a_np = np.random.uniform(0, 1, (128,)).astype("float32")
b_np = np.random.uniform(0, 1, (128,)).astype("float32")
c_np = a_np + b_np
a_tvm = tvm.nd.array(a_np)
b_tvm = tvm.nd.array(b_np)
c_tvm = tvm.nd.array(np.empty((128,), dtype=np.float32))

In [16]:
mod = tvm.build(ir, target="llvm")
mod["add"](a_tvm, b_tvm, c_tvm)
np.testing.assert_allclose(c_tvm.numpy(), c_np, rtol=1e-5)